# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [1]:
#!pip install -U --pre tensorflow=="2.*"

Make sure you have `pycocotools` installed

In [1]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [2]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [4]:
# # %%bash
# !cd models/research/
# !protoc object_detection\protos\*.proto --python_out=.


In [5]:
# %%bash 
# python
!cd models/research
# cp object_detection/packages/tf2/setup.py .
!pip3 install . --upgrade
# !pip install .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


### Imports

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [4]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

from object_detection.utils import np_box_list_ops
from object_detection.utils import np_box_list
from object_detection.utils import np_box_ops
from object_detection.core import box_list

Patches:

In [6]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [7]:
def load_model(model_name):
  #base_url = 'http://download.tensorflow.org/models/object_detection/'
  #model_file = model_name + '.tar.gz'
#   model_dir = tf.keras.utils.get_file(
#     fname=model_name, 
#     origin=base_url + model_file,
#     untar=True)

  model_dir = pathlib.Path(model_name)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model



In [50]:
%%bash
pwd
# cd models/research/object_detection
# pwd


/mnt/c/Thesis/Thesis/DigitPathology/tensorflow-detection/test results


## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [51]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = r'C:/Thesis/Thesis/DigitPathology/tensorflow-detection/data/object-detection.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

Load an object detection model:

In [52]:
model_name = r'C:\Thesis\Thesis\DigitPathology\tensorflow-detection\hand_labels_groundtruth_model\KP_Groundtruth_Graph_full'
# model_name = 'KP_Gaze_Graph'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [53]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


And returns several outputs:

In [54]:
detection_model.output_dtypes

{'raw_detection_boxes': tf.float32,
 'detection_scores': tf.float32,
 'raw_detection_scores': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32}

In [55]:
detection_model.output_shapes

{'raw_detection_boxes': TensorShape([None, 300, 4]),
 'detection_scores': TensorShape([None, 300]),
 'raw_detection_scores': TensorShape([None, 300, 2]),
 'detection_multiclass_scores': TensorShape([None, 300, 2]),
 'detection_classes': TensorShape([None, 300]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 300, 4])}

Add a wrapper function to call the model, and cleanup the outputs:

In [56]:
import pathlib
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [63]:
## Malisiewicz et al.
def non_max_suppression_fast(detection_matrix, overlapThresh):
    # if there are no boxes, return an empty list
    if len(detection_matrix) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if detection_matrix.dtype.kind == "i":
        detection_matrix = detection_matrix.astype("float")
    # initialize the list of picked indexes	
    pick = []
    output = []

    
    # return boxes with score != 100
    scores = detection_matrix[:,4].tolist()
    scores = [round(100*score) for score in scores]
    pick1 = [i for i, n in enumerate(scores) if n != 100]
#     print(pick1)

    if pick1: 
        for i in pick1:
            print(i)
            output.append(detection_matrix[i].tolist())
    
    detection_matrix = np.delete(detection_matrix, pick1, 0) 
   
    
    # grab the coordintes of the bounding boxes
    x1 = detection_matrix[:,0]
    y1 = detection_matrix[:,1]
    x2 = detection_matrix[:,2]
    y2 = detection_matrix[:,3]

    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap >= overlapThresh)[0])))

# return only the bounding boxes that were picked 
    for j in pick:
        output.append(detection_matrix[j].tolist())
    return np.array(output)


In [64]:
import pandas as pd

In [101]:
def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)

    #_________________________EDITED FOR NMS WHEN SCORES = 100%_____________________________
    min_score_thresh = 0.96   
    #send only those indices that have score == 100%~~~~~~~~
    scores = []
    boxes = []
    classes = []  
    detections = len(output_dict['detection_scores']) 

    for i in range(detections):
     if output_dict['detection_scores'][i] > 0.9:    #min_score_thresh:
        scores.append(output_dict['detection_scores'][i])
        boxes.append(output_dict['detection_boxes'][i])
        classes.append(output_dict['detection_classes'][i])

    boxes = np.array(boxes) 
    scores = np.array([scores])
    classes = np.array([classes])
#     print("Boxes:",len(boxes)) 
    
    results_ = None
    if len(boxes) == 0:
        return pd.DataFrame()
   

    detection_matrix = np.hstack((np.hstack((boxes,scores.T)),classes.T))
    output = non_max_suppression_fast(detection_matrix, 1.0)

    scores = output[:,4]
    classes = output[:,5].astype(np.int)
    boxes = output[:,0:4]

    #________________________________________________________________________________

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
    #       output_dict['detection_boxes'],
    #       output_dict['detection_classes'],
    #       output_dict['detection_scores'],
      boxes, classes, scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=12,
      min_score_thresh=min_score_thresh)

    #Save detections in a csv
    height = image_np.shape[0]
    width =  image_np.shape[1] 

    if len(boxes) != 1:
        boxes = np.squeeze(boxes) 
    for box in boxes:
        box[0] = np.round(box[0] * height).astype(np.int)
        box[1] = np.round(box[1] * width).astype(np.int)
        box[2] = np.round(box[2] * height).astype(np.int)
        box[3] = np.round(box[3] * width).astype(np.int)

    results_ =   np.column_stack((boxes, scores, classes))
    #   print(results_)
    img_name = str(image_path).split("\\")[-1]

    results_ = pd.DataFrame(results_, columns=['xmin','ymin','xmax','ymax','score','class'], index = None)
    results_.insert(0, "image_name", [img_name] * len(results_))


    #   np.savetxt( pathlib.Path(image_path).stem+'gaze_result.csv',
    #              np.column_stack((boxes, scores, classes)), fmt = '%s', header="xmin,ymin,xmax,ymax,score,class",
    #              delimiter=',',comments='')

#     Save out/put image
    im= Image.fromarray(image_np)
    im.save(str(pathlib.Path(image_path).parent)+'\\'+ str(pathlib.Path(image_path).stem) +'_gaze_result_full.jpg')  
#     print(str(pathlib.Path(image_path).parent)+'\\'+ str(pathlib.Path(image_path).stem) +'_gaze_result_full.jpg')

#     display(Image.fromarray(image_np))
    return results_

In [102]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path(r"C:\Thesis\Thesis\DigitPathology\tensorflow-detection\faster_rcnn_gaze_output")
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
len(TEST_IMAGE_PATHS)

os.makedirs(os.path.join(PATH_TO_TEST_IMAGES_DIR,"output_folder"), exist_ok=True) 


In [103]:
COLUMN_NAMES=['image_name','xmin','ymin','xmax','ymax','score','class']
df_results = pd.DataFrame(columns=COLUMN_NAMES)
#run test on image one at a time. Do not give multiple images.
# image_path = r"C:\Thesis\Thesis\DigitPathology\tensorflow-detection\validation_set\images_jpg\29-17-IIDC.svs_[4000,0,4000,4000]_9.jpg"
# results = show_inference(detection_model, image_path)
# df_results = pd.concat([df_results ,results])
# print(df_results)
# df_results.to_csv(os.path.join(PATH_TO_TEST_IMAGES_DIR, "Output.csv"), index = False)

for jpg_path in TEST_IMAGE_PATHS:
#     print(jpg_path)
    results = show_inference(detection_model, jpg_path)
    if not results.empty:
        df_results = pd.concat([df_results ,results])
# print(df_results)
# df_results.to_csv(os.path.join(PATH_TO_TEST_IMAGES_DIR, "Output.csv"), index = False)


C:\Users\KOMAL~1.MAR\AppData\Local\Temp/ipykernel_19308/1205237319.py:36: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  classes = output[:,5].astype(np.int)
C:\Users\KOMAL~1.MAR\AppData\Local\Temp/ipykernel_19308/1205237319.py:61: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check t

10
11
1
2
